In [8]:
"""
now it works on all cities together, calculate all_cities_zscore for each study region
"""

import geopandas as gpd
import pandas as pd
import os
import time
import sv_config as sc
import sv_setup_hexes_allCities as ssh

startTime = time.time()
dirname = os.path.abspath('')
gpkg_input = os.path.join(dirname, sc.output_cities)
gdf_layers = []
layers = []
for i in list(sc.layerNames.values()):
    try:
        gdf = gpd.read_file(gpkg_input, layer=i)
        gdf = gdf.reindex(columns=sorted(gdf.columns))
        gdf_layers.append(gdf)
        layers.append(i)
    except ValueError as e:
        print(e)

# create a big dataframe(all_cities_hex_df) according to gdf_layers
all_cities_hex_df = pd.concat(gdf_layers, ignore_index=True)

Null layer: 'melbourne'
Null layer: 'hex_temp'


In [9]:
all_cities_hex_df

,Study region,geometry,pop_daily_living,pop_nh_intersection_density,pop_nh_pop_density,pop_pct_access_500m_convenience,pop_pct_access_500m_pt_any,pop_pct_access_500m_public_open_space,pop_pct_access_500m_supermarkets,pop_walkability,urban_sample_point_count
0,Odense,"MULTIPOLYGON (((589115.578 6145014.850, 589126...",2.321481,121.900978,2483.269751,36.216761,62.426846,96.633922,36.870572,0.376287,50707
1,odense_sample,"POLYGON ((587615.233 6140232.460, 587659.042 6...",3.735216,219.704253,4961.054066,87.140845,99.260063,99.925679,87.195040,-0.174561,708


In [10]:
fieldNames = [
    sc.city_fieldNames['pop_nh_pop_density'],
    sc.city_fieldNames['pop_nh_intersection_density'],
    sc.city_fieldNames['pop_daily_living']
]

fieldNames_new = [
    sc.city_fieldNames['all_cities_pop_z_nh_population_density'],
    sc.city_fieldNames['all_cities_pop_z_nh_intersection_density'],
    sc.city_fieldNames['all_cities_pop_z_daily_living']
]

fieldNames_zip = list(zip(fieldNames, fieldNames_new))

for index, layer in enumerate(gdf_layers):
    for field_zip in fieldNames_zip:
        mean, std = ssh.getMeanStd(all_cities_hex_df, field_zip[0])
        layer[field_zip[1]] = (layer[field_zip[0]] - mean) / std
    # sum all three new fields
    layer[sc.hex_fieldNames['all_cities_walkability']] = layer[fieldNames_new].sum(axis=1)
    layer.to_file(gpkg_input, layer=layers[index], driver='GPKG')

In [13]:
gdf_layers[0].columns

Index(['Study region', 'geometry', 'pop_daily_living',
       'pop_nh_intersection_density', 'pop_nh_pop_density',
       'pop_pct_access_500m_convenience', 'pop_pct_access_500m_pt_any',
       'pop_pct_access_500m_public_open_space',
       'pop_pct_access_500m_supermarkets', 'pop_walkability',
       'urban_sample_point_count', 'all_cities_pop_z_nh_population_density',
       'all_cities_pop_z_nh_intersection_density',
       'all_cities_pop_z_daily_living', 'all_cities_walkability'],
      dtype='object')

In [14]:
gdf_layers[1].columns

Index(['Study region', 'geometry', 'pop_daily_living',
       'pop_nh_intersection_density', 'pop_nh_pop_density',
       'pop_pct_access_500m_convenience', 'pop_pct_access_500m_pt_any',
       'pop_pct_access_500m_public_open_space',
       'pop_pct_access_500m_supermarkets', 'pop_walkability',
       'urban_sample_point_count', 'all_cities_pop_z_nh_population_density',
       'all_cities_pop_z_nh_intersection_density',
       'all_cities_pop_z_daily_living', 'all_cities_walkability'],
      dtype='object')